In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.getenv('SAM_PATH'))
from samutil import MultiThreading
from tqdm.notebook import tqdm
import json

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = '03ba24122703f9cff69789b9ebbd342e'

In [3]:
c = tmdb.Discover()

In [11]:
results = []
years_check = [1980]
for i in years_check:
    total_pages_to_loop = c.movie(year=i,page=1)['total_pages']
    for loop in range(1,total_pages_to_loop):
        movie = c.movie(year=i,page=loop)
        results.append(movie['results'])
        


In [16]:
results[0]

[{'adult': False,
  'backdrop_path': '/AdKA2F1SzYPhSZdEbjH1Zh75UVQ.jpg',
  'genre_ids': [27, 53],
  'id': 694,
  'original_language': 'en',
  'original_title': 'The Shining',
  'overview': "Jack Torrance accepts a caretaker job at the Overlook Hotel, where he, along with his wife Wendy and their son Danny, must live isolated from the rest of the world for the winter. But they aren't prepared for the madness that lurks within.",
  'popularity': 50.841,
  'poster_path': '/nRj5511mZdTl4saWEPoj9QroTIu.jpg',
  'release_date': '1980-05-23',
  'title': 'The Shining',
  'video': False,
  'vote_average': 8.2,
  'vote_count': 13010},
 {'adult': False,
  'backdrop_path': '/lDxGguxqFy1ch8XElYaNXqw0CuN.jpg',
  'genre_ids': [27, 53, 12],
  'id': 578,
  'original_language': 'en',
  'original_title': 'Jaws',
  'overview': 'When an insatiable great white shark terrorizes the townspeople of Amity Island, the police chief, an oceanographer and a grizzled shark hunter seek to destroy the blood-thirsty bea

In [8]:
df_1000 = pd.read_csv('top_1000_films.csv',encoding='utf-8')

In [9]:
df_1000

,movie,gross,year
0,Avatar,2847246203,2009
1,Avengers: Endgame,2797501328,2019
2,Titanic,2201647264,1997
3,Star Wars: Episode VII - The Force Awakens,2069521700,2015
4,Avengers: Infinity War,2048359754,2018
...,...,...,...
995,Jackass 3D,171685792,2010
996,No Country for Old Men,171627166,2007
997,Spies in Disguise,171616764,2019
998,John Wick: Chapter 2,171547802,2017


In [10]:
def film_switch(dic,x):
    try:
        return dic[x]
    except:
        return x

replace_dic = {'Star Wars: Episode VII - The Force Awakens': 'Star Wars: The Force Awakens',
'Star Wars: Episode VI - Return of the Jedi' :'Return of the Jedi',
'Star Wars: Episode VIII - The Last Jedi': 'Star Wars: The Last Jedi',
'Spider-Man: Far from Home': 'Spider-Man: Far From Home',
'Star Wars: Episode IX - The Rise of Skywalker': 'Star Wars: The Rise of Skywalker',
"Harry Potter and the Sorcerer's Stone": "Harry Potter and the Philosopher's Stone",
"Hi, Mom":'Hi, Mom!',
"Star Wars: Episode IV - A New Hope": 'Star Wars - A New Hope',
"Star Wars: Episode V - The Empire Strikes Back": 'The Empire Strikes Back',
"Once Upon a Time... In Hollywood": 'Once Upon a Time… in Hollywood',
"Die Hard with a Vengeance": 'Die Hard: With a Vengeance',
"Fantastic 4: Rise of the Silver Surfer": 'Fantastic Four: Rise of the Silver Surfer',
"Indiana Jones and the Raiders of the Lost Ark":'Raiders of the Lost Ark',            
"The 40-Year-Old Virgin": 'The 40 Year Old Virgin',}

# replace_dic = dict((v,k) for k,v in replace_dic.items())
df_1000.movie = df_1000.movie.apply(lambda x: film_switch(replace_dic,x))

# Get Movie Id
---

In [11]:
def get_response(search_query):
    search = tmdb.Search()
    response = search.movie(query=search_query)
    return response

In [12]:
def get_film_id(search_query):
    search = tmdb.Search()
    response = search.movie(query=search_query)
    return response['results'][0]['id']

In [13]:
def get_film_response(film_id):
    film_response = tmdb.Movies(film_id)
    return film_response

In [14]:
def build_film_dic(film_response):
    film_dic = {}
    film_dic['title'] = film_response.info()['title']
    film_dic['id'] = film_response.id
    film_dic['release_date'] = film_response.release_date
    film_dic['overview'] = film_response.info()['overview']
    film_dic['budget'] = film_response.budget
    film_dic['genres'] = film_response.genres
    film_dic['vote_average'] = film_response.vote_average
    film_dic['vote_count'] = film_response.vote_count
    film_dic['reviews'] = film_response.reviews()['results']
    film_dic['keywords'] = film_response.keywords()['keywords']
    film_dic['production_companies'] = film_response.info()['production_companies']
    film_dic['production_countries'] = film_response.info()['production_countries']

    return film_dic

In [15]:
results = []

In [16]:
def get_film_details(search_queries):
    
    for query in tqdm(search_queries):
        try:
            print('0')
            film_id = get_film_id(query)
            print('a')
            film_response = get_film_response(film_id)
            print('b')
            film_dic = build_film_dic(film_response)
            print('c')
            results.append(film_dic)
        except:
            print('pass')
            print(query)
            
    return results

In [17]:
mt = MultiThreading(15,df_1000.movie.to_list(),output=results)

In [18]:
mt.Run(get_film_details)

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

...

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
aaa
ba
b
a
b

b


a
b
a
b
a
b
b
a
b
a
b
a
b
a
b
aa
b

b
a
b
cc
0

c
0
c
0
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
cc

0
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
aa
b

b
a
b
a
b
a
b
a
b
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
a
b
c
0
a
b
a
b
a
b
cc

0
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
cc
0

c
cc
c
c
c
00

c

c
0
0
0
c
0
0
0
0
0
c
0
c
c0

0
c
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
c
0
a
b
c
0
c
0
c
0
c
0
c
c
0
0
c
0
cc

0
c
0
0
c
0
c
0
c
0
c
0
a
b
a
b
aaaa
b

b

b

b
a
ba
b
a

b
a
b
a
b
a
a
b
b
a
b
c
0
a
b
c
0
c
0
c
0
c
0
c
c
0
0
c
0
c
c
c
0
0
0
c
0
c
0
c
0
c
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
c
0
a
b
c
c0

0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
c
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
c
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
c
0
a
b
c
0
c
0
c
c
0
0
c
0
c
0
cc
0

0
c
0
a
b
c
0
a
b
c
0
c
0
c
0
c
0
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
a
b
c
0
c
0
c
0
a
b
c
0
c
0
c
0
c
0
c
0
c
0
c
0
a
b
a
b
c
0
c
0
c
0
a
b
c


In [19]:
df_tmdb = pd.DataFrame(results)
# df_tmdb.columns = ['movie','id','release_date','description','popularity','released','vote']

In [20]:
df_tmdb = df_tmdb.drop_duplicates(subset=['title'],keep='first')

In [21]:
df_tmdb

,title,id,release_date,overview,budget,genres,vote_average,vote_count,reviews,keywords,production_companies,production_countries
0,Ice Age: The Meltdown,950,2006-03-29,"Diego, Manny and Sid return in this sequel to ...",80000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",6.6,7667,[],"[{'id': 2078, 'name': 'mammoth'}, {'id': 2079,...","[{'id': 11749, 'logo_path': '/5AzTeZkSCFhiGZ86...","[{'iso_3166_1': 'US', 'name': 'United States o..."
1,Harry Potter and the Chamber of Secrets,672,2002-11-13,"Cars fly, trees fight back, and a mysterious h...",100000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.7,16556,[],"[{'id': 391, 'name': 'flying car'}, {'id': 616...","[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'..."
2,The Addams Family,481084,2019-10-10,The Addams family's lives begin to unravel whe...,40000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",6.6,1455,"[{'author': 'SWITCH.', 'author_details': {'nam...","[{'id': 4553, 'name': 'black humor'}, {'id': 6...","[{'id': 75234, 'logo_path': None, 'name': 'Cin...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
3,Godzilla vs. Kong,399566,2021-03-24,"In a time when monsters walk the Earth, humani...",200000000,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",8.0,6053,"[{'author': 'SWITCH.', 'author_details': {'nam...","[{'id': 11100, 'name': 'giant monster'}, {'id'...","[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'US', 'name': 'United States o..."
4,The Boss Baby: Family Business,459151,2021-07-02,The Templeton brothers — Tim and his Boss Baby...,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",0.0,0,[],"[{'id': 1009, 'name': 'baby'}, {'id': 9663, 'n...","[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
994,Erin Brockovich,462,2000-03-17,A twice-divorced mother of three who sees an i...,52000000,"[{'id': 18, 'name': 'Drama'}]",7.4,2207,"[{'author': 'John Chard', 'author_details': {'...","[{'id': 5565, 'name': 'biography'}, {'id': 967...","[{'id': 216, 'logo_path': None, 'name': 'Jerse...","[{'iso_3166_1': 'US', 'name': 'United States o..."
995,Man of Steel,49521,2013-06-12,A young boy learns that he has extraordinary p...,225000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",6.6,12188,"[{'author': 'KCruz88', 'author_details': {'nam...","[{'id': 83, 'name': 'saving the world'}, {'id'...","[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'..."
996,As Good as It Gets,2898,1997-12-19,"New York City. Melvin Udall, a cranky, bigoted...",50000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",7.4,2666,[],"[{'id': 641, 'name': 'single parent'}, {'id': ...","[{'id': 18, 'logo_path': '/uqhagSwM7NAkQWVCQhK...","[{'iso_3166_1': 'US', 'name': 'United States o..."
997,Hotel Transylvania 3: Summer Vacation,400155,2018-06-28,"Dracula, Mavis, Johnny and the rest of the Dra...",80000000,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",6.9,3129,"[{'author': 'John', 'author_details': {'name':...","[{'id': 1299, 'name': 'monster'}, {'id': 3133,...","[{'id': 5, 'logo_path': '/71BqEFAF4V3qjjMPCpLu...","[{'iso_3166_1': 'US', 'name': 'United States o..."


# Get Films Missed
---

In [22]:
a = build_film_dic(get_film_response(get_response('Spider-Man 2')['results'][1]['id']))
b = build_film_dic(get_film_response(get_response('Guardians of the Galaxy')['results'][1]['id']))
c = build_film_dic(get_film_response(get_response('Justice League')['results'][0]['id']))
d = build_film_dic(get_film_response(get_response('Men in Black')['results'][1]['id']))
e = build_film_dic(get_film_response(get_response('Ted')['results'][1]['id']))
f = build_film_dic(get_film_response(get_response('Cars')['results'][1]['id']))
g = build_film_dic(get_film_response(get_response('Return of the Jedi')['results'][0]['id']))
h = build_film_dic(get_film_response(get_response('Home Alone')['results'][5]['id']))
i = build_film_dic(get_film_response(get_response('X2')['results'][1]['id']))
j = build_film_dic(get_film_response(get_response('Mission: Impossible')['results'][1]['id']))
k = build_film_dic(get_film_response(get_response('Indiana Jones and the Raiders')['results'][1]['id']))
l = build_film_dic(get_film_response(get_response('the Mummy')['results'][1]['id']))
m = build_film_dic(get_film_response(get_response('Superman')['results'][1]['id']))
n = build_film_dic(get_film_response(get_response('Creed')['results'][1]['id']))
o = build_film_dic(get_film_response(get_response('Fast & Furious')['results'][2]['id']))
p = build_film_dic(get_film_response(get_response('Top Gun')['results'][0]['id']))



In [23]:
results_new = [a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p]

In [24]:
df_tmdb_edited = pd.concat([df_tmdb,pd.DataFrame(results_new)])

In [25]:
df_merge = pd.merge(df_tmdb_edited,df_1000,left_on='title',right_on='movie',how='outer')


In [26]:
df_merge = df_merge.drop_duplicates(subset=['id'],keep='first')

In [27]:
df_merge = df_merge.dropna()

In [28]:
df_merge = df_merge.set_index('title')

# Cast and Crew Info

In [29]:
film_dic = dict(zip(df_tmdb_edited.id,df_tmdb_edited.movie))

AttributeError: 'DataFrame' object has no attribute 'movie'

In [30]:
cast_output = []
crew_output = []

In [31]:
def get_cast(movie_iteration):
    
    for i in tqdm(movie_iteration):
        try:
            movie = tmdb.Movies(i)

            cast_output.append((i,movie.credits()['cast']))

        except:
            pass
            
    return
        

In [32]:
def get_crew(movie_iteration):
    
    for i in tqdm(movie_iteration):
        try:
            movie = tmdb.Movies(i)

            crew_output.append((i,movie.credits()['crew']))    
        except:
            pass
            
    return
        

In [33]:
mt_cast = MultiThreading(15,df_merge['id'].to_list(),output=cast_output)
mt_cast.Run(get_cast)

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

...

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

In [34]:
mt_cast = MultiThreading(15,df_merge['id'].to_list(),output=crew_output)
mt_cast.Run(get_crew)

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

...

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

In [35]:
df_cast = pd.DataFrame(cast_output)
df_cast.columns = ['id','cast']
df_cast = df_cast.set_index('id')

In [36]:
df_crew = pd.DataFrame(crew_output)
df_crew.columns = ['id','crew']
df_crew = df_crew.set_index('id')

In [73]:
df_final = pd.concat([df_merge.reset_index().set_index('id'),df_cast,df_crew],axis=1)

In [74]:
df_final = df_final.reset_index()

In [75]:
df_final.drop(columns='movie',inplace=True)

In [76]:
def shrink_dictionary(x,key):
    genres = []
    for dic in x:
        genres.append(dic[key])
    return list(set(genres))

df_final['genres'] = df_final['genres'].apply(lambda x: shrink_dictionary(x,'name'))
df_final['keywords'] = df_final['keywords'].apply(lambda x: shrink_dictionary(x,'name'))
df_final['production_countries'] = df_final['production_countries'].apply(lambda x: shrink_dictionary(x,'name'))
df_final['production_companies'] = df_final['production_companies'].apply(lambda x: shrink_dictionary(x,'name'))
# df_final['cast_name'] = df_final['cast'].apply(lambda x: shrink_dictionary(x,'name'))
# df_final['crew_name'] = df_final['crew'].apply(lambda x: shrink_dictionary(x,'name'))

In [77]:
df_final['absolute_profit'] = df_final['gross'] - df_final['budget']
df_final['relative_profit'] = df_final['absolute_profit'] / df_final['budget']

In [79]:
df_final = df_final[['id', 'title', 'release_date', 'overview', 'budget','gross','absolute_profit','relative_profit','genres',
       'vote_average', 'vote_count', 'reviews', 'keywords',
       'production_companies', 'production_countries', 'year', 'cast',
       'crew']]

In [80]:
df_final.to_json('imdb_film_data.json')